In [1]:
import numpy as np
import open3d as o3d
import pprint as pp
import matplotlib.pyplot as plt
import pyransac3d as pyrsc

In [2]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    #outが外れ値
    print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    return inlier_cloud

def voxcel_down(pcd,v_size):
    print("Downsample the point cloud with a voxel of %s"%v_size)
    downpcd = pcd.voxel_down_sample(voxel_size=v_size)
    return downpcd

def remove_wall(downpcd, t):
    #壁の削除
    points = np.asarray(downpcd.points)
    # 平面モデルを定義
    plano1 = pyrsc.Plane()
    # RANSACによる平面推定。平面の式とインライア(inlier)を計算。しきい値は0.01
    best_eq, best_inliers = plano1.fit(points, t)
    # 元のデータにおけるインライアの点の色を変更
    plane = downpcd.select_by_index(best_inliers).paint_uniform_color([1, 0, 0])
    # 平面以外の点を抽出
    not_plane = downpcd.select_by_index(best_inliers, invert=True)
    return not_plane, plane

def remove_desk(not_plane,t):
    #机の削除
    desk_points = np.asarray(not_plane.points)
    # 平面モデルを定義
    plano1 = pyrsc.Plane()
    # RANSACによる平面推定。平面の式とインライア(inlier)を計算。しきい値は0.01
    # 机の部分を削除（多分閾値でいけるはず）
    best_eq, desk_inliers = plano1.fit(desk_points, t)
    # 元のデータにおけるインライアの点の色を変更（赤色に変更）
    desk_place = not_plane.select_by_index(desk_inliers).paint_uniform_color([1, 0, 0])
    # 平面以外の点を抽出
    desk_not_plane = not_plane.select_by_index(desk_inliers, invert=True)
    return desk_not_plane, desk_place

def farthest_point_sample(point, npoint):
    """
    Input:
        xyz: pointcloud data, [N, D]
        npoint: number of samples
    Return:
        centroids: sampled pointcloud index, [npoint, D]
    """
    N, D = point.shape
    xyz = point[:,:3]
    centroids = np.zeros((npoint,))
    distance = np.ones((N,)) * 1e10
    farthest = np.random.randint(0, N)
    for i in range(npoint):
        centroids[i] = farthest
        centroid = xyz[farthest, :]
        dist = np.sum((xyz - centroid) ** 2, -1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance, -1)
    point = point[centroids.astype(np.int32)]
    return point

In [3]:
#ファイル名指定（plyファイル）
pcd = o3d.io.read_point_cloud("frame.1-4.s303.df1.5.ply")
#voxel_sizeを指定
v_size = 0.01
#voxcel_downsampling
downpcd = voxcel_down(pcd, v_size)

print("\nOriginal bounds")
pp.pprint(downpcd.get_min_bound())
pp.pprint(downpcd.get_max_bound())

boundingbox = o3d.geometry.AxisAlignedBoundingBox(
    min_bound=(-0.53,  -0.5,   -1.49),
    max_bound=( 0.62,   0.58,  -0.49))
cropped_pcd = downpcd.crop(boundingbox)

print("\nNew bounds")
pp.pprint(cropped_pcd.get_min_bound())
pp.pprint(cropped_pcd.get_max_bound())

o3d.visualization.draw_geometries([cropped_pcd,])

Downsample the point cloud with a voxel of 0.01

Original bounds
array([-0.73673183, -0.50005886, -1.49950004])
array([ 0.81960982,  0.5861842 , -0.49050003])

New bounds
array([-0.52989908, -0.49947567, -1.48965389])
array([ 0.61996696,  0.44768266, -0.49050003])
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
#閾値設定
threshold = 0.01
#remove wall
not_plane, plane = remove_wall(downpcd, threshold)
pp.pprint(not_plane)
#3次元座標上の値を出力
# pp.pprint(np.asarray(not_plane.points))
#可視化
o3d.visualization.draw_geometries([not_plane, plane])

PointCloud with 52329 points.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
#外れ値除去をやる
#statistical
print("Radius oulier removal")
cl, ind = not_plane.remove_statistical_outlier(nb_neighbors=30,std_ratio=2.0)
processed_point = display_inlier_outlier(not_plane, ind)

Radius oulier removal
Showing outliers (red) and inliers (gray): 


In [6]:
#閾値設定
desk_threshold = 0.02
#机の削除
desk_not_plane, desk_plane = remove_desk(processed_point, desk_threshold)
pp.pprint(desk_not_plane)

PointCloud with 294325 points.


In [8]:
#外れ値除去をやる
#radisu
print("Radius oulier removal")
cl, ind = desk_not_plane.remove_radius_outlier(nb_points=20, radius=0.05)
processed_point = display_inlier_outlier(desk_not_plane, ind)
# o3d.visualization.draw_geometries([processed_point])

Radius oulier removal
Showing outliers (red) and inliers (gray): 


In [9]:
# pp.pprint(np.asarray(processed_point.points))
#FPSを実施
human_point = np.asarray(processed_point.points)
#array形式に変更されている
fps = farthest_point_sample(human_point, len(human_point)//30)
#numpy→open3d型に変更
main_pcd = o3d.geometry.PointCloud()
main_pcd.points = o3d.utility.Vector3dVector(fps)
pp.pprint(main_pcd)

PointCloud with 9809 points.


In [10]:
# 可視化
o3d.visualization.draw_geometries([main_pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
